<a href="https://colab.research.google.com/github/motallebimasoud-cmyk/binance-volume-bot/blob/main/Daily_2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
import datetime

# -------------------------------
# تنظیمات تلگرام
# -------------------------------
TELEGRAM_TOKEN = "8421847126:AAF8DyYSO76IBzQ8IY7tV7SC0qzzcT6-T20"
CHAT_ID = "953826363"

# -------------------------------
# تنظیمات فیلتر (قابل تغییر)
# -------------------------------
PRICE_CHANGE_MIN = 3       # حداقل درصد رشد قیمت (مثلاً 3٪)
VOLUME_MULTIPLIER = 2.5    # چند برابر شدن حجم نسبت به میانگین (مثلاً 2.5 برابر)

def send_telegram(message: str):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    try:
        requests.post(url, data={
            "chat_id": CHAT_ID,
            "text": message,
            "parse_mode": "HTML"   # برای ارسال تمیزتر
        })
    except Exception as e:
        print("خطا در ارسال پیام:", e)

# -------------------------------
# گرفتن لیست جفت ارزهای USDT از کوکوین
# -------------------------------
def get_usdt_pairs():
    url = "https://api.kucoin.com/api/v1/symbols"
    r = requests.get(url).json()
    pairs = []
    for s in r["data"]:
        if s["quoteCurrency"] == "USDT" and s["enableTrading"]:
            symbol = s["symbol"]
            # حذف توکن‌های لوریج‌دار (3L, 3S, UP, DOWN)
            if any(x in symbol for x in ["3L", "3S", "UP", "DOWN"]):
                continue
            pairs.append(symbol)
    return pairs

# -------------------------------
# گرفتن کندل روزانه ۲۰ روز آخر
# -------------------------------
def get_klines(symbol: str):
    url = f"https://api.kucoin.com/api/v1/market/candles?type=1day&symbol={symbol}"
    r = requests.get(url).json()
    if r.get("data"):
        return r["data"][:21]  # ۲۰ روز گذشته + امروز
    return []

# -------------------------------
# فیلتر حجم غیرعادی + رشد قیمت
# -------------------------------
def check_symbol(symbol: str):
    data = get_klines(symbol)
    if len(data) < 21:
        return None

    # ساختار دیتا: [time, open, close, high, low, volume, turnover]
    today = data[0]
    past20 = data[1:]

    today_open = float(today[1])
    today_close = float(today[2])
    today_volume = float(today[5])

    avg_volume_20 = sum(float(d[5]) for d in past20) / len(past20)
    price_change = (today_close - today_open) / today_open * 100

    if price_change > PRICE_CHANGE_MIN and today_volume > VOLUME_MULTIPLIER * avg_volume_20:
        return (
            f"🚀 <b>سیگنال KuCoin</b>\n"
            f"📊 جفت‌ارز: <b>{symbol}</b>\n"
            f"📈 رشد: <b>{price_change:.2f}%</b>\n"
            f"🔹 حجم امروز: <b>{today_volume:.2f}</b>\n"
            f"🔹 میانگین ۲۰ روز: <b>{avg_volume_20:.2f}</b>"
        )
    return None

# -------------------------------
# اجرای ربات
# -------------------------------
def run():
    pairs = get_usdt_pairs()
    for symbol in pairs:
        try:
            msg = check_symbol(symbol)
            if msg:
                send_telegram(msg)
                print("ارسال شد:", symbol)
        except Exception as e:
            print("خطا در", symbol, ":", e)

# اجرای هر ۲۴ ساعت
if __name__ == "__main__":
    while True:
        print("شروع بررسی:", datetime.datetime.now())
        run()
        print("پایان - صبر تا فردا")
        time.sleep(24 * 60 * 60)  # هر روز یک بار

شروع بررسی: 2025-09-22 17:39:13.657526
ارسال شد: SUN-USDT
ارسال شد: ALPHA-USDT
ارسال شد: LOOKS-USDT
ارسال شد: SWEAT-USDT
ارسال شد: LAVA-USDT
ارسال شد: TUT-USDT
ارسال شد: GORK-USDT
ارسال شد: NAM-USDT
پایان - صبر تا فردا
